### Plotting Test
Alex is using this Jupyter notebook to test the plotting + print statements from query.py.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lsst.rsp import get_tap_service
import query_helpers as q

##### Testing object classifcation
There are two ways this classification function works:
1. Providing orbit parameters, returning a corresponding object type
2. Providing an object type, reutrning corresponding orbit parameters

In [ ]:
# 1
input_params = {
        "q_cutoff_min": None, 
        "q_cutoff": None,
        "a_cutoff_min": None, 
        "a_cutoff": None,
        "e_cutoff_min": None, 
        "e_cutoff": None,
        "t_cutoff_min": 2.0,
        "t_cutoff": 3.0}

object_type = q.type_classification(input_params = input_params)
print(object_type)

#2
params = q.type_classification(object_type = "JFC")
print(params)

Now that know our classification works, let's try making a full query. 

In [ ]:
# testing with NEOs
query, object_type = q.make_query_general(object_type = "NEO")
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "NEO", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(q_cutoff=1.3, a_cutoff=4.0, e_cutoff = 1.0, join = 'Diasource')
print(query)
print(object_type) 


In [ ]:
# running the query
NEO_objects_table = q.run_query(query)

In [ ]:
# Adding calculated + object_type columns to data table
a = q.calc_semimajor_axis(NEO_objects_table['q'], NEO_objects_table['e'])
NEO_objects_table['a'] = a
NEO_objects_table['object_type'] = object_type

print(NEO_objects_table[0:5]) # print first few rows 
print(NEO_objects_table.columns)

In [ ]:
q.plot_data(NEO_objects_table)

# # Manual Plotting Code 
fig, ax = plt.subplots()
# plt.xlim([0., 4.])
# plt.ylim([0., 1.])
ax.scatter(NEO_objects_table["a"], NEO_objects_table["e"], s=0.1) # a vs. i
# ax.set_xscale('log')
ax.set_xlabel('semimajor axis (au)')
ax.set_ylabel('eccentricity')
ax.set_title("a vs. e (NEO)")
ax.minorticks_on()
ax.grid()
plt.show()

In [ ]:
# q.type_counts(NEO_objects_table)
print(NEO_objects_table.columns)
df = NEO_objects_table.to_pandas()
print(df['object_type'].value_counts())
type(df['object_type'].value_counts())

#### How many observations for each object? In what filters?

In [ ]:
# # need to count observations for each unique object in SSO_id
# print(df['ssObjectID'].value_counts())
# # unique observations within each filter
# print(df['band'].value_counts())
# # count of unique observations for each unique object in SSO_id within each filter
# temp = df.groupby(['ssObjectID', 'band']).size().reset_index(name='obs_filter_count')
# print(df.groupby(['ssObjectID', 'band']).size().reset_index(name='obs_filter_count'))
# print(temp.columns)

observations_by_object_filter = q.obs_filter(df)

#### What is the average magnitude range? Does any object have an unusually large range?

Want to group everything by its object type, then group the observations by object, then want to get the min/max brightnesses from each object, then want to get average min/max values, then if something is more than one std.dev away, want to call it out. 

In [ ]:
df.columns

In [ ]:
"""
Function groups everything by object type, observations by unique object, gets min/max mags
"""

# 1. Group observations by object type, by ssObjectID, get the min/max/mean magnitudes
grouped_obs_data = df.groupby(['object_type', 'ssObjectID']).agg(
    mag_min = ('magTrueVband', 'min'), 
    mag_max = ('magTrueVband', 'max'), 
    mag_mean = ('magTrueVband', 'mean')
)
grouped_obs_data = grouped_obs_data.reset_index() # groupby turns 'object_type' and 'ssObjectID' into indeces, this turns them back into columns
# print(grouped_obs_data) # degbugging

# 2. Create ranges column from min/max magnitudes, get standard deviation of ranges. 
# want to look at when the range has a larger spread than normal, so want to get the std. deviation of the range
mag_range = grouped_obs_data["mag_max"] - grouped_obs_data["mag_min"]
# print(mag_range) # debugging
grouped_obs_data['mag_range'] = mag_range
print("Standard Deviation of Range:", np.std(mag_range))
print("Mean Range:", np.mean(mag_range))

# 3. Need to check which ranges are above change criterion. Using mean and std. deviation by object_type. 
large_criterion = np.mean(mag_range) + (np.std(mag_range)*2)
print(f"Large range criterion:", large_criterion)
filtered_large_ranges = grouped_obs_data[grouped_obs_data['mag_range'] > large_criterion]
# print(filtered_large_ranges) # debugging
# print(filtered_large_ranges.columns) #debugging

# 4. Rearranging dataframe so magnitude ranges are in descending order (largest at the top).
sorted_filt_lrg_ranges = filtered_large_ranges.sort_values(by='mag_range', ascending=False)
# print(sorted_filt_lrg_ranges) # debugging

In [ ]:
# getting top 10 ranges
top_srtd_filt_lrg_ranges = sorted_filt_lrg_ranges.iloc[:20,:]
top_ranges = top_srtd_filt_lrg_ranges.copy()
top_ranges['y_spacing'] = range(1, len(top_ranges) + 1) 
# print(top_ranges) #degbugging


# For the largest objects, want to visualize their ranges
fig, ax = plt.subplots()
ax.hlines(data = top_ranges, y = 'y_spacing', xmin = 'mag_min', xmax = 'mag_max', color = "black", linewidth = 2, zorder = 2, label = None)

# adding reference line
x_center = (top_ranges['mag_min'].min() + top_ranges['mag_max'].max()) / 2
xmin_ref = x_center - (np.mean(mag_range/2))
xmax_ref = x_center + (np.mean(mag_range/2))
ax.hlines(y = 0, xmin = xmin_ref, xmax = xmax_ref, color = "blue", linewidth = 2, label = "Mean Range") #want this to be in the center of the plot
# print("Mean Range:", np.mean(mag_range))
sc = ax.scatter(data = top_ranges, x = 'mag_mean', y = 'y_spacing', s = 40, marker='o', edgecolors = "black", linewidth = 0.5, c='mag_mean', cmap = 'PiYG', zorder = 3, label = None)
cbar = fig.colorbar(sc, ax=ax)
cbar.set_label('Mean Magnitude (V)')
ax.set_xlabel('Magnitude Range (V)')
ax.set_ylabel('ssObject ID')
ax.set_title("Top 20 Magnitude Ranges")
ax.set_yticks(top_ranges['y_spacing'])
ax.set_yticklabels(top_ranges['ssObjectID'])
ax.minorticks_on()
ax.grid(zorder = 1)
plt.legend(loc="lower left")
plt.show()


# plt.scatter('x', 'y', s='size', c='density', data=df, alpha=0.5, cmap='Blues')

In [ ]:
# just plotting all of the NEOs above 2 sigma
data_table = sorted_filt_lrg_ranges

top_srtd_filt_lrg_ranges = sorted_filt_lrg_ranges.iloc[:10,:]
top_ranges = data_table.copy()
top_ranges['y_spacing'] = range(1, len(top_ranges) + 1) 
# print(top_ranges) #degbugging

color_map = {
    "NEO": "red",
    "TNO": "blue",
    "Centaur": "green",
    "MBA": "orange",
    "Jtrojan": "purple",
    "LPC": "brown",
    "Ntrojan": "pink"
}

color = color_map.get(object_type, "gray")  # fallback to 'gray' if type unknown
# For the largest objects, want to visualize their ranges
fig, ax = plt.subplots()
ax.hlines(data = top_ranges, y = 'y_spacing', xmin = 'mag_min', xmax = 'mag_max', color = color, linewidth = 2, zorder = 2, label = object_type)

# adding reference line
x_center = (top_ranges['mag_min'].min() + top_ranges['mag_max'].max()) / 2
xmin_ref = x_center - (np.mean(mag_range/2))
xmax_ref = x_center + (np.mean(mag_range/2))
ax.hlines(y = 0, xmin = xmin_ref, xmax = xmax_ref, color = "blue", linewidth = 2, label = "Mean Range") #want this to be in the center of the plot
# print("Mean Range:", np.mean(mag_range))
sc = ax.scatter(data = top_ranges, x = 'mag_mean', y = 'y_spacing', s = 40, marker='o', edgecolors = "black", linewidth = 0.5, c='mag_mean', cmap = 'PiYG', zorder = 3, label = None)
cbar = fig.colorbar(sc, ax=ax)
cbar.set_label('Mean Magnitude (V)')
ax.set_xlabel('Magnitude Range (V)')
ax.set_ylabel('ssObject ID')
ax.set_title("Magnitude Ranges")
# ax.set_yticks(top_ranges['y_spacing'])
# ax.set_yticklabels(top_ranges['ssObjectID'])
ax.minorticks_on()
ax.grid(zorder = 1)
plt.legend(loc="lower left")
plt.show()

In [ ]:
# sorted_filt_lrg_ranges.columns

# q.mag_range_plot(sorted_filt_lrg_ranges, number = 5)
# q.mag_range_plot(sorted_filt_lrg_ranges, number = 40)
# q.mag_range_plot(sorted_filt_lrg_ranges, number = None)